## Implementation of Parallel MMDE
The following code implements the P-MMDE. It includes the test functions and the scipy nested parallel DE.
First we include the relevant libraries.

In [1]:
# nested DE for global optimization for a minmax objective function
import json
import time
import numpy as np

from scipy.optimize import differential_evolution
genLL = 10
npopLL = 10
genUL = 5
npopUL = 20

Then we define the optimization problems, including their bounds and their upper and lower level objective functions.

In [ ]:
# define the bounds on the search
def get_bounds(t):
    r_minL, r_maxL = 0.0, 10.0
    # define the bounds on the LL search
    boundsLL = [(r_minL, r_maxL)] * t
    r_minU, r_maxU = 0.0, 10.0
    # define the bounds on the ul search
    boundsUL = [(r_minU, r_maxU)] * t
    return boundsLL, boundsUL

In [4]:
# f1 objective (equivelant to f8)
# optimal solution: (5,5) optimal obj: 0
def f1UL(x, t):
    resultll = differential_evolution(f1LL, bounds=get_bounds(t)[0], maxiter=genLL, popsize=npopLL, workers=1, args=(x, t))
    return -resultll.fun


def f1LL(x, y, t):
    f1 = sum((x[t] - 5) ** 2 - (y[t] - 5) ** 2 for t in range(t)) 
    return f1

In [5]:
# f2 objective (equivelant to f9)
# optimal solution: (0,0) optimal obj: 3
def f2UL(x, t):
    resultll = differential_evolution(f2LL, bounds=get_bounds(t)[0],  maxiter=genLL, popsize=npopLL, workers=1, args=(x, t),disp=False)
    return -resultll.fun


def f2LL(x, y, t):
    f2_1 = 3 + sum(- 0.2 * x[t] + 0.3 * y[t] for t in range(t))
    f2_2 = 3 + sum(0.2 * x[t] - 0.1 * y[t] for t in range(t))
    f2 = min(f2_1, f2_2)
    return -f2

In [6]:
# f3 objective (equivelant to f10)
# optimal solution: (10,2.1257) optimal obj: 0.097794

def f3UL(x, t):
    resultll = differential_evolution(f3LL, bounds=get_bounds(t)[0],  maxiter=genLL, popsize=npopLL, workers=1, args=(x, t),disp=False)
    return -resultll.fun


def f3LL(x, y, t):
    f3 = np.divide(sum(np.sin(x[t]-y[t])for t in range(t)),sum(np.sqrt(x[t]**2+y[t]**2)for t in range(t))) #send it as f3
    return f3

In [7]:
# f4 objective (equivelant to f11)
# optimal solution: (7.0441,10) optimal obj: 0.042488

def f4UL(x, t):
    resultll = differential_evolution(f4LL, bounds=get_bounds(t)[0], maxiter=genLL, popsize=npopLL, workers=1, args=(x, t),disp=False)
    return -resultll.fun


def f4LL(x, y, t):
    f4 = np.divide(sum(np.cos(np.sqrt(x[t]**2+y[t]**2))for t in range(t)),sum((np.sqrt(x[t]**2+y[t]**2)+10) for t in range(t)))
    #f = np.divide(sum(np.cos(np.sqrt(x[t]**2+y[t]**2))for t in range(t)),sum(np.sqrt(x[t]**2+y[t]**2)+10 for t in range(t))
    return -f4

Then we run the experiments for different number of cores, different dimensionality of the problem and a number of runs. We then save the results in a txt file.

In [ ]:
if __name__ == "__main__":
    tfunction = 1
    for core in [1,2,4,8,16,24]: #removed the 1/add it again
        for t in [1,2,5,10]:
            rt = []
            for i in range(0, 20):
                start = time.time()
                result = differential_evolution(f1UL, bounds=get_bounds(t)[1],  strategy='best1bin',maxiter=genUL, workers=core,
                                                popsize=npopUL, args=(t,), updating='deferred',disp=False)
                                                #updating='deferred', args=(t,))
                end = time.time()
                rt.append((core, end - start, result.x.tolist(), result.fun.tolist()))
                #print('Runtime %s cores:  %s' % (core, end - start))
                #print('Runtime %s result x:  %s' % (core, result.x))
                #print('Runtime %s result f:  %s' % (core, result.fun))
            with open(f'results_fun{tfunction}_dim{t}_cores{core}.txt', 'w') as f:
                f.write(json.dumps(rt))
                